In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import os

notebook_path = os.path.abspath('')
import sys
# Find the part of the path that contains 'commitFit'
commit_fit_path = None
for part in notebook_path.split(os.sep):
    print(part)
    if 'ensemble_commit' in part:
        commit_fit_path = notebook_path.split(part)[0] + part
        print(commit_fit_path)
        break

if commit_fit_path is None:
    raise ValueError("Path containing 'ensemble_commit' not found in notebook path.")


if commit_fit_path not in sys.path:
    sys.path.append(commit_fit_path)

import ensemble_model.preprocesser as preprocesser 
import ensemble_model.MoE_model_loss as moe 
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer, RobertaModel, RobertaTokenizer, DistilBertModel

# import whatthepatch
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_curve,classification_report



root
ensemble_commit
/root/ensemble_commit


In [2]:
moe.BiLSTMStudent

ensemble_model.MoE_model_loss.BiLSTMStudent

In [3]:
df = pd.read_csv(r'../datasets/dataset.csv', encoding='utf_8_sig')
df.dropna(inplace=True)
label2id={'negative':0,'positive':1}
df = df.replace({"label": label2id})
df

/tmp/ipykernel_6699/1052552286.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"label": label2id})


,Unnamed: 0,label,message,diff
0,0,0,Merge pull request #46 from rufferson/saslx-tl...,diff --git a/lib/DJabberd.pm b/lib/DJabberd.pm...
1,1,1,Fix leaks in kadmin server stubs [CVE-2015-863...,diff --git a/src/kadmin/server/server_stubs.c ...
2,2,1,Validate authorization request on approval\n\n...,diff --git a/spring-security-oauth2/src/main/j...
3,3,1,Release 2.72.4+171110,diff --git a/application/config/version.php b/...
4,4,0,Fixing compiler warnings.,diff --git a/src/main.c b/src/main.c\nindex 50...
...,...,...,...,...
10107,10107,0,les: remove useless protocol defines (#22115)\...,diff --git a/les/benchmark.go b/les/benchmark....
10108,10108,1,Merge pull request #2067 from realm/tg-swift-l...,diff --git a/CHANGELOG.md b/CHANGELOG.md\ninde...
10109,10109,0,[fix] 新規アカウント作成ページの翻訳を修正,diff --git a/app/locales/ja_JP/LC_MESSAGES/mes...
10110,10110,0,Merge branch 'hotfixes',diff --git a/CHANGELOG.txt b/CHANGELOG.txt\nin...


In [4]:
# df['command'] = df['diff'].apply(lambda x : extract_lines(find_longest_list([y.changes for y in whatthepatch.parse_patch(x)])))
# extracted_cols = df['diff'].str.extract(r'^diff --git (?P<source>[^\t\n]+) (?P<target>[^\t\n]+)')
# whatthepatch.apply_diff(diff, lao)

In [5]:
# len(df_dataset)

In [6]:
# df_dataset[0]

In [7]:
# Load BERT and CodeBERT models and tokenizers
bert_model = BertModel.from_pretrained('../models/bert-base-cased')
bert_tokenizer = BertTokenizer.from_pretrained('../models/bert-base-cased')

codebert_model = BertModel.from_pretrained('../models/bert-base-cased')
codebert_tokenizer = BertTokenizer.from_pretrained('../models/bert-base-cased')

Some weights of the model checkpoint at ../models/bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at ../models/bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 

In [8]:
train_data, test_data = train_test_split(df, test_size=0.3, random_state=42)
test_data, val_data = train_test_split(test_data, test_size=0.5, random_state=42)
# train_data = pd.read_csv('./datasets/PD_train.csv')
# test_data = pd.read_csv('./datasets/PD_test.csv')
# val_data = pd.read_csv('./datasets/PD_val.csv')



In [9]:
# codebert_model.config.hidden_size

In [10]:
len(test_data)

1510

In [11]:
train_data,_ = train_test_split(train_data, train_size=0.1, random_state=42)
test_data, val_data = train_test_split(test_data, train_size=0.1, random_state=42)
val_data, _ = train_test_split(val_data, train_size=0.1, random_state=42)

In [12]:
train_data.reset_index(inplace=True)
test_data.reset_index(inplace=True)
val_data.reset_index(inplace=True)

In [13]:
train_data['label'].value_counts()

label
0    448
1    256
Name: count, dtype: int64

In [14]:
val_data

,index,Unnamed: 0,label,message,diff
0,8267,8267,0,Merge branch 'cassandra-3.0' into cassandra-3.11,diff --git a/CHANGES.txt b/CHANGES.txt\nindex ...
1,5734,5734,0,Fix getPicon,diff --git a/plugin/controllers/models/service...
2,1397,1397,0,1.0.6,diff --git a/package.json b/package.json\ninde...
3,8156,8156,0,Added envconf() wrapper around conf(),diff --git a/lib/Functions.php b/lib/Functions...
4,7255,7255,0,Update VERSION to 3.0.0,diff --git a/CHANGES b/CHANGES\nindex 86460f0f...
...,...,...,...,...,...
130,41,41,0,Merge pull request #870 from AshKyd/develop\n\...,diff --git a/php/Admin/Settings/OverallPanel.p...
131,5434,5434,0,Send Async,diff --git a/plugin/Live/on_publish.php b/plug...
132,6889,6889,0,Update translations from Crowdin for pl_PL,diff --git a/modules/po/controlpanel.pl_PL.po ...
133,4097,4097,1,XSS 취약점 수정,diff --git a/adm/boardgroup_form.php b/adm/boa...


In [15]:
val_data

,index,Unnamed: 0,label,message,diff
0,8267,8267,0,Merge branch 'cassandra-3.0' into cassandra-3.11,diff --git a/CHANGES.txt b/CHANGES.txt\nindex ...
1,5734,5734,0,Fix getPicon,diff --git a/plugin/controllers/models/service...
2,1397,1397,0,1.0.6,diff --git a/package.json b/package.json\ninde...
3,8156,8156,0,Added envconf() wrapper around conf(),diff --git a/lib/Functions.php b/lib/Functions...
4,7255,7255,0,Update VERSION to 3.0.0,diff --git a/CHANGES b/CHANGES\nindex 86460f0f...
...,...,...,...,...,...
130,41,41,0,Merge pull request #870 from AshKyd/develop\n\...,diff --git a/php/Admin/Settings/OverallPanel.p...
131,5434,5434,0,Send Async,diff --git a/plugin/Live/on_publish.php b/plug...
132,6889,6889,0,Update translations from Crowdin for pl_PL,diff --git a/modules/po/controlpanel.pl_PL.po ...
133,4097,4097,1,XSS 취약점 수정,diff --git a/adm/boardgroup_form.php b/adm/boa...


In [16]:


# Create Datasets and DataLoaders
train_dataset = preprocesser.SentencePairDataset(train_data, bert_tokenizer, codebert_tokenizer, message='message',command='diff',label='label')
val_dataset = preprocesser.SentencePairDataset(val_data, bert_tokenizer, codebert_tokenizer, message='message',command='diff',label='label')
test_dataset = preprocesser.SentencePairDataset(test_data, bert_tokenizer, codebert_tokenizer, message='message',command='diff',label='label')

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)



# # Initialize the model
# model = em.CombinedModel(bert_model, codebert_model, bert_tokenizer, codebert_tokenizer)

# # Train the model
# model.trainer(train_loader, val_loader,num_epochs=10)
# for batch in train_loader:
#     print(batch)

In [17]:
base_model1 = moe.BaseModel(bert_model)
base_model2 = moe.BaseModel(codebert_model)

In [18]:
# Load BERT and CodeBERT models and tokenizers
bert_model = DistilBertModel.from_pretrained('/root/autodl-tmp/IPCK/model/distilbert')

codebert_model = DistilBertModel.from_pretrained('/root/autodl-tmp/IPCK/model/distilbert')

Some weights of the model checkpoint at /root/autodl-tmp/IPCK/model/distilbert were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at /root/autodl-tmp/IPCK/model/distilbert were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you

In [19]:
encoder_model1 = moe.EncoderModel(bert_model)
encoder_model2 = moe.EncoderModel(bert_model)

In [20]:


# Create stacking model
moe_model = moe.MoEModel(base_model1, base_model2)
# Train the model
moe_model.trainer(train_loader, val_loader,num_epochs=30, patience=3)

/root/miniconda3/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/30 Loss: 0.1637: 100%|██████████| 88/88 [01:47<00:00,  1.22s/batch]


=============================train========================


Epoch 2/30 Loss: 0.1449: 100%|██████████| 88/88 [02:01<00:00,  1.38s/batch]


=============================train========================


Epoch 3/30 Loss: 0.1285: 100%|██████████| 88/88 [01:46<00:00,  1.20s/batch]


...

=============================train========================


Epoch 12/30 Loss: 0.0379: 100%|██████████| 88/88 [01:40<00:00,  1.15s/batch]


=============================train========================


Epoch 13/30 Loss: 0.0472: 100%|██████████| 88/88 [01:41<00:00,  1.15s/batch]


=============================train========================
Early stopping triggered after 13 epochs
Validation Accuracy: 0.7407
Precision: 0.7612
Recall: 0.7407
F1-Score: 0.7441


In [25]:
test_dataset = preprocesser.SentencePairDataset(test_data, bert_tokenizer, codebert_tokenizer,message='message',command='diff',label='label')
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [26]:
torch.save(moe_model, "entire_bert_model_900repo.pth")

In [27]:
moe_model = torch.load("entire_bert_model_900repo.pth")                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [28]:
test_acc, test_labels, test_probabilities, test_embeddings, test_predictions = moe_model.evaluate(test_loader)

Validation Accuracy: 0.8477
Precision: 0.8572
Recall: 0.8477
F1-Score: 0.8494


In [29]:
print(classification_report(test_labels,test_predictions))

              precision    recall  f1-score   support

           0       0.92      0.83      0.87        94
           1       0.76      0.88      0.81        57

    accuracy                           0.85       151
   macro avg       0.84      0.85      0.84       151
weighted avg       0.86      0.85      0.85       151



In [30]:
test_acc

0.847682119205298

In [31]:
model_save_path = 'best_model'

In [32]:
student_model = moe.BiLSTMStudent(hidden_dim=256,output_dim =2, base_model1=encoder_model1,base_model2=encoder_model2)

In [33]:
# train_student_model

In [34]:
student_model.distill_trainer(moe_model, test_loader, num_epochs=10)

/root/miniconda3/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/10 Loss: 2.1954: 100%|██████████| 19/19 [00:15<00:00,  1.22batch/s]


Epoch 1/10, Training Loss: 2.1954


Epoch 2/10 Loss: 1.9120: 100%|██████████| 19/19 [00:15<00:00,  1.19batch/s]


Epoch 2/10, Training Loss: 1.9120


Epoch 3/10 Loss: 1.1175: 100%|██████████| 19/19 [00:15<00:00,  1.23batch/s]


...

Epoch 8/10, Training Loss: 0.1235


Epoch 9/10 Loss: 0.0523: 100%|██████████| 19/19 [00:15<00:00,  1.21batch/s]


Epoch 9/10, Training Loss: 0.0523


Epoch 10/10 Loss: 0.0282: 100%|██████████| 19/19 [00:14<00:00,  1.33batch/s]


Epoch 10/10, Training Loss: 0.0282
Validation Accuracy: 0.8477
Precision: 0.8572
Recall: 0.8477
F1-Score: 0.8494


In [42]:
test_acc, test_labels, test_probabilities, test_embeddings, test_predictions = student_model.evaluate(val_loader)

Validation Accuracy: 0.6963
Precision: 0.7170
Recall: 0.6963
F1-Score: 0.7003


In [43]:
print(classification_report(test_labels,test_predictions))

              precision    recall  f1-score   support

           0       0.80      0.67      0.73        83
           1       0.58      0.73      0.65        52

    accuracy                           0.70       135
   macro avg       0.69      0.70      0.69       135
weighted avg       0.72      0.70      0.70       135



In [39]:
test_acc, test_labels, test_probabilities, test_embeddings, test_predictions = moe_model.evaluate(val_loader)

Validation Accuracy: 0.7407
Precision: 0.7612
Recall: 0.7407
F1-Score: 0.7441


In [40]:
print(classification_report(test_labels,test_predictions))

              precision    recall  f1-score   support

           0       0.84      0.71      0.77        83
           1       0.63      0.79      0.70        52

    accuracy                           0.74       135
   macro avg       0.74      0.75      0.74       135
weighted avg       0.76      0.74      0.74       135



In [41]:
torch.save(student_model, "student_model_900repo.pth")